## Consider the College data from the ISLP package. Details about the data is described on page 65 of the ISLP textbook for this class (https://islp.readthedocs.io/en/latest/datasets/College.html). We would like to predict the number of applications received using the other variables. 80% of the data (randomly generated) will be treated as training data. The rest will be the test data.

In [11]:
%%capture
# importlib.util allows us to check whether a package is already installed
import importlib.util

# Check if the ISLP package is available in the current Jupyter kernel
if importlib.util.find_spec("ISLP") is None:
    # Install ISLP if it is not already installed
    !pip install ISLP

### a) Fit a linear model using least squares and report the estimate of the test error.

In [12]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import ModelSpec as MS
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# load college dataset
college = load_data("College")

# Split the data into 80% training and 20% test sets
train_df, test_df = train_test_split(college, test_size=0.2, random_state=42)

# Fit a linear model using least squares (OLS)
# Remove response from predictors
predictors = train_df.columns.drop("Apps")

design = MS(predictors)
design = design.fit(train_df)

# Create the design matrices for training and testing data
X_train = design.transform(train_df)
y_train = train_df["Apps"]
X_test = design.transform(test_df)
y_test = test_df["Apps"]

# Fit the OLS model using statsmodels
model = sm.OLS(y_train, X_train)
results = model.fit()

# Report the estimate of the test error (Mean Squared Error)
# Make predictions on the test set
predictions = results.predict(X_test)

# Calculate the Mean Squared Error (MSE) on the test set
test_error = mean_squared_error(y_test, predictions)

#print(f"Estimated test MSE: {test_error:.2f}")


### b) Fit a tree to the data. Summarize the results. Unless the number of terminal nodes is large, display the tree graphically. Report its MSE.

In [13]:
from sklearn.tree import DecisionTreeRegressor, plot_tree

# Encode categorical variable
college = pd.get_dummies(college, columns=["Private"], drop_first=True)
college["Private_Yes"] = college["Private_Yes"].astype(int)

# Split predictors and response
X = college.drop("Apps", axis=1)
y = college["Apps"]

# Train–test split (80% / 20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Fit a regression tree
#    max_depth=3 keeps the tree interpretable
# tree_model = DecisionTreeRegressor(
#     max_depth=3,
#     random_state=42
# )
# tree_model.fit(X_train, y_train)

tree_model = DecisionTreeRegressor(
    ccp_alpha=0.0,
    random_state=42
)
tree_model.fit(X_train, y_train)


# Predict and compute test MSE
y_pred = tree_model.predict(X_test)
tree_mse = mean_squared_error(y_test, y_pred)

# print(f"Regression Tree Test MSE: {tree_mse:,.2f}")

# Display the tree graphically
# plt.figure(figsize=(20, 10))
# plot_tree(
#     tree_model,
#     feature_names=X.columns.tolist(),
#     filled=True,
#     rounded=True,
#     impurity=False,
#     label="all",
#     node_ids=True,
#     fontsize=10
# )

#plt.title("Regression Tree (Max Depth = 3)")
#plt.savefig("Regression Tree.png", dpi=300, bbox_inches="tight")
#plt.show()


### c) Use Cross validation to determine whether pruning is helpful and determine the optimal size for the pruned tree. Compare the pruned and un-pruned trees. Report MSE for the pruned tree. Which predictors seem to be the most important?

In [14]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor

# Encode categorical variable
# college = pd.get_dummies(college, columns=["Private"], drop_first=True)
# college["Private_Yes"] = college["Private_Yes"].astype(int)

# X = college.drop("Apps", axis=1)
# y = college["Apps"]

# # Train–test split (80% / 20%)
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42
# )

# Cost-complexity pruning path
base_tree = DecisionTreeRegressor(random_state=42)
path = base_tree.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas = path.ccp_alphas

# Cross-validation to select optimal ccp_alpha
param_grid = {"ccp_alpha": ccp_alphas}

grid = GridSearchCV(
    DecisionTreeRegressor(random_state=42),
    param_grid,
    cv=5,
    scoring="neg_mean_squared_error"
)
grid.fit(X_train, y_train)

optimal_ccp_alpha = grid.best_params_["ccp_alpha"]
#print(f"Optimal ccp_alpha: {optimal_ccp_alpha}")

# Fit un-pruned tree (baseline)
unpruned_tree = DecisionTreeRegressor(ccp_alpha=0.0, random_state=42)
unpruned_tree.fit(X_train, y_train)

y_pred_unpruned = unpruned_tree.predict(X_test)
mse_unpruned = mean_squared_error(y_test, y_pred_unpruned)

# Fit pruned tree (CV-selected)
pruned_tree = DecisionTreeRegressor(
    ccp_alpha=optimal_ccp_alpha,
    random_state=42
)
pruned_tree.fit(X_train, y_train)

y_pred_pruned = pruned_tree.predict(X_test)
mse_pruned = mean_squared_error(y_test, y_pred_pruned)

# Report MSEs
# print(f"\nUn-pruned Tree Test MSE: {mse_unpruned:,.2f}")
# print(f"Pruned Tree Test MSE:   {mse_pruned:,.2f}")

# Important predictors (from pruned tree)
feature_importance = (
    pd.Series(pruned_tree.feature_importances_, index=X.columns)
      .sort_values(ascending=False)
)

# print("\nMost Important Predictors (Pruned Tree):")
# print(feature_importance.head())


### d) Use a bagging approach to analyze the data with B = 500 and B = 1000. Compute the MSE. Which predictors seem to be the most important?

In [15]:
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

# The variable 'Private' is categorical (Yes/No).
# For tree-based methods, it is sufficient to encode it as a binary variable (0/1).
# college["Private"] = college["Private"].astype("category").cat.codes


# Define predictors and response
X = college.drop("Apps", axis=1)   # Predictor variables
y = college["Apps"]                # Response variable
feature_names = X.columns          # Save feature names for interpretation


# Train–test split (80% training, 20% test)
# A fixed random_state ensures reproducibility
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


# Bagging function
# This function fits a BaggingRegressor with a specified
# number of trees (n_estimators) and returns the test MSE.
def evaluate_bagging(n_estimators, X_train, y_train, X_test, y_test):
    
    # Bagging uses bootstrap samples and averages predictions
    # max_samples=1.0 means each bootstrap sample has the same size as the training set
    bag_model = BaggingRegressor(
        estimator=DecisionTreeRegressor(),  # base learner
        n_estimators=n_estimators,           # number of trees
        max_samples=1.0,                     # bootstrap sample size
        bootstrap=True,                      # sample with replacement
        random_state=42,
        n_jobs=-1                            # use all available CPU cores
    )
    
    # Fit the bagging model
    bag_model.fit(X_train, y_train)
    
    # Predict on the test data
    y_pred = bag_model.predict(X_test)
    
    # Compute test Mean Squared Error
    mse = mean_squared_error(y_test, y_pred)
    
    return mse, bag_model


# Evaluate bagging with different numbers of trees
# Compare performance as the number of trees increases
mse_500, model_500 = evaluate_bagging(
    500, X_train, y_train, X_test, y_test
)
# print(f"Test MSE with B = 500 trees: {mse_500:.2f}")

mse_1000, model_1000 = evaluate_bagging(
    1000, X_train, y_train, X_test, y_test
)
# print(f"Test MSE with B = 1000 trees: {mse_1000:.2f}")


# Identify important predictors
rf_bagging = RandomForestRegressor(
    n_estimators=1000,
    max_features=None,   # use all predictors at each split (bagging)
    random_state=42,
    n_jobs=-1
)

rf_bagging.fit(X_train, y_train)


# Feature importance
feature_importance = pd.DataFrame(
    {"importance": rf_bagging.feature_importances_},
    index=feature_names
).sort_values(by="importance", ascending=False)

# print("\nMost Important Predictors:")
# print(feature_importance)

# # Plot feature importance
# feature_importance.plot(kind="bar", legend=False)
# plt.title("Feature Importance for Predicting College Applications")
# plt.ylabel("Importance (Mean Decrease in Impurity)")
# plt.tight_layout()
# plt.show()


### e) Repeat (d) with a random forest approach with B = 500 and B = 1000, and m ≈ p = 3.

In [16]:
from sklearn.ensemble import RandomForestRegressor
import math

# # Load the College data
# college = load_data("College")

# # Convert 'Private' (Yes/No) to numeric (0/1)
# college["Private"] = college["Private"].factorize()[0]

# # Define predictors and response
# X = college.drop("Apps", axis=1)
# y = college["Apps"]

# # Train–test split (80% / 20%)
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42
# )

# Set Random Forest parameters
p = X_train.shape[1]     # number of predictors
m = 3                   # user-specified m ≈ p/3

# print(f"Number of predictors (p): {p}")
# print(f"Using m = {m} predictors at each split.\n")

# Function to train and evaluate Random Forest
def train_and_evaluate_rf(n_estimators, max_features):
    
    rf_model = RandomForestRegressor(
        n_estimators=n_estimators,   # B
        max_features=max_features,   # m
        bootstrap=True,
        random_state=42,
        n_jobs=-1
    )
    
    rf_model.fit(X_train, y_train)
    
    y_pred = rf_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    
    # print(f"Random Forest (B = {n_estimators}, m = {max_features})")
    # print(f"  Test MSE : {mse:,.2f}")
    # print(f"  Test RMSE: {rmse:,.2f}\n")
    
    return rf_model, mse


# Fit models for B = 500 and B = 1000
# rf_500, mse_500 = train_and_evaluate_rf(500, m)
# rf_1000, mse_1000 = train_and_evaluate_rf(1000, m)


## 2. Consider the business school admission data available in the admission.csv. The admission officer of a business school has used an “index” of undergraduate grade point average (GPA,𝑋1) and graduate management aptitude test (GMAT,𝑋2) scores to help decide which applicants should be admitted to the school’s graduate programs. This index is used to categorize each applicant into one of three groups – admit (group 1), do not admit (group 2), and borderline (group 3). We will take the last four observations in each category as test data and the remaining observations as training data.

### a) Perform an exploratory analysis of the training data by examining appropriate plots and comment on how helpful these predictors may be in predicting response.

In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
admission = pd.read_csv("admission.csv")

# Split into training and test data
# last 4 observations in each group are test data

train_parts = []
test_parts = []

for _, group_df in admission.groupby("Group"):
    train_parts.append(group_df.iloc[:-4])
    test_parts.append(group_df.iloc[-4:])

train_df = pd.concat(train_parts).reset_index(drop=True)
test_df = pd.concat(test_parts).reset_index(drop=True)

# print(f"Training data shape: {train_df.shape}")
# print(f"Test data shape: {test_df.shape}")

# Summary statistics (training data only)
# print("\nSummary statistics (training data):")
# print(train_df.groupby("Group")[["GPA", "GMAT"]].describe())

# Scatter plot: GPA vs GMAT
# plt.figure(figsize=(6, 4))
# sns.scatterplot(
#     data=train_df,
#     x="GPA",
#     y="GMAT",
#     hue="Group",
#     palette="Set1",
#     s=40
# )
# plt.title("Training Data: GPA vs GMAT by Admission Group")
# plt.xlabel("Undergraduate GPA (X₁)")
# plt.ylabel("GMAT Score (X₂)")
# plt.grid(True)
# # save
# plt.savefig("scatter_gpa_gmat.jpg", dpi=300, bbox_inches="tight")
# plt.show()

# # Boxplots
# fig, axes = plt.subplots(1, 2, figsize=(8, 4))

# sns.boxplot(data=train_df, x="Group", y="GPA", ax=axes[0])
# axes[0].set_title("GPA by Admission Group")

# sns.boxplot(data=train_df, x="Group", y="GMAT", ax=axes[1])
# axes[1].set_title("GMAT by Admission Group")
# # save
# plt.savefig("boxplots_gpa_gmat.jpg", dpi=300, bbox_inches="tight")
# plt.show()


### b) Perform an LDA using the training data. Superimpose the decision boundary on an appropriate display of the data. Does the decision boundary seem sensible? In addition, compute the confusion matrix and overall misclassification rate based on both training and test data. What do you observe?

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, accuracy_score

# Load the data
df = pd.read_csv("admission.csv")

# Split into training and test data
# last 4 observations in each group are test data
train_parts = []
test_parts = []

for _, group_df in df.groupby("Group"):
    train_parts.append(group_df.iloc[:-4])
    test_parts.append(group_df.iloc[-4:])

df_train = pd.concat(train_parts).reset_index(drop=True)
df_test = pd.concat(test_parts).reset_index(drop=True)

X_train = df_train[["GPA", "GMAT"]]
y_train = df_train["Group"]

X_test = df_test[["GPA", "GMAT"]]
y_test = df_test["Group"]

# Perform LDA
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)

# Plot decision boundary
def plot_decision_boundary(X, y, model, title):
    h = 0.02
    x_min, x_max = X["GPA"].min() - 0.5, X["GPA"].max() + 0.5
    y_min, y_max = X["GMAT"].min() - 50, X["GMAT"].max() + 50

    xx, yy = np.meshgrid(
        np.arange(x_min, x_max, h),
        np.arange(y_min, y_max, h)
    )

    grid = pd.DataFrame(
        np.c_[xx.ravel(), yy.ravel()],
        columns=["GPA", "GMAT"]
    )

    Z = model.predict(grid)
    Z = Z.reshape(xx.shape)

    # plt.contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.Paired)
    # plt.scatter(
    #     X["GPA"], X["GMAT"],
    #     c=y.astype(int),
    #     edgecolors="k",
    #     cmap=plt.cm.Paired
    # )
    # plt.xlabel("GPA")
    # plt.ylabel("GMAT")
    # plt.title(title)
    # plt.savefig("LDA_Decision.jpg", dpi=300, bbox_inches="tight")
    # plt.show()


# Training decision boundary
plot_decision_boundary(X_train, y_train, lda, "LDA Decision Boundary (Training Data)")

# Confusion matrix and accuracy
y_pred_train = lda.predict(X_train)
y_pred_test = lda.predict(X_test)

# print("Confusion Matrix (Test Data):")
# print(confusion_matrix(y_test, y_pred_test))

# print("\nAccuracy (Train):", accuracy_score(y_train, y_pred_train))
# print("Accuracy (Test):", accuracy_score(y_test, y_pred_test))
# print("Misclassification Rate (Test):", 1 - accuracy_score(y_test, y_pred_test))


### c) Repeat (b) using QDA.

In [19]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# Perform QDA
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train, y_train)

# Plot QDA decision boundary
def plot_decision_boundary(X, y, model, title):
    h = 0.02
    x_min, x_max = X["GPA"].min() - 0.5, X["GPA"].max() + 0.5
    y_min, y_max = X["GMAT"].min() - 50, X["GMAT"].max() + 50

    xx, yy = np.meshgrid(
        np.arange(x_min, x_max, h),
        np.arange(y_min, y_max, h)
    )

    grid = pd.DataFrame(
        np.c_[xx.ravel(), yy.ravel()],
        columns=["GPA", "GMAT"]
    )

    Z = model.predict(grid)
    Z = Z.reshape(xx.shape)

    # plt.contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.Paired)
    # plt.scatter(
    #     X["GPA"], X["GMAT"],
    #     c=y.astype(int),
    #     edgecolors="k",
    #     cmap=plt.cm.Paired
    # )
    # plt.xlabel("GPA")
    # plt.ylabel("GMAT")
    # plt.title(title)
    # plt.savefig("QDA_Decision.jpg", dpi=300, bbox_inches="tight")
    # plt.show()

# Plot training decision boundary
plot_decision_boundary(
    X_train, y_train, qda,
    "QDA Decision Boundary (Training Data)"
)

# Confusion matrix and accuracy
y_pred_train = qda.predict(X_train)
y_pred_test = qda.predict(X_test)

# print("Confusion Matrix (Test Data):")
# print(confusion_matrix(y_test, y_pred_test))

# print("\nAccuracy (Train):", accuracy_score(y_train, y_pred_train))
# print("Accuracy (Test):", accuracy_score(y_test, y_pred_test))
# print("Misclassification Rate (Test):", 1 - accuracy_score(y_test, y_pred_test))


### d) Fit a KNN with K chosen optimally using test error rate. Report error rate, sensitivity, specificity, and AUC for the optimal KNN based on the training data. Also, report its estimated test error rate.

In [20]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler

# Train / test split
test_data = pd.concat([
    df[df["Group"] == 1].tail(4),
    df[df["Group"] == 2].tail(4),
    df[df["Group"] == 3].tail(4)
])

train_data = df.drop(test_data.index)

X_train = train_data[["GPA", "GMAT"]]
y_train = train_data["Group"]

X_test = test_data[["GPA", "GMAT"]]
y_test = test_data["Group"]

# Standardize predictors (required for KNN)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Choose optimal K using TEST error rate
k_range = range(1, 15)
test_errors = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    y_test_pred = knn.predict(X_test_scaled)
    test_errors.append(1 - accuracy_score(y_test, y_test_pred))

optimal_k = k_range[np.argmin(test_errors)]
# print(f"Optimal K (chosen by test error): {optimal_k}")

# Fit optimal KNN
knn_opt = KNeighborsClassifier(n_neighbors=optimal_k)
knn_opt.fit(X_train_scaled, y_train)

# TRAINING DATA performance
y_train_pred = knn_opt.predict(X_train_scaled)
y_train_probs = knn_opt.predict_proba(X_train_scaled)

train_error = 1 - accuracy_score(y_train, y_train_pred)

cm_train = confusion_matrix(y_train, y_train_pred)

# Sensitivity (Recall) per class
sensitivity = np.diag(cm_train) / cm_train.sum(axis=1)

# Specificity per class
specificity = []
for i in range(cm_train.shape[0]):
    tn = cm_train.sum() - (cm_train[i, :].sum() + cm_train[:, i].sum() - cm_train[i, i])
    fp = cm_train[:, i].sum() - cm_train[i, i]
    specificity.append(tn / (tn + fp))

sensitivity_macro = np.mean(sensitivity)
specificity_macro = np.mean(specificity)

auc_train = roc_auc_score(y_train, y_train_probs, multi_class="ovr")

# print("\n--- TRAINING DATA PERFORMANCE ---")
# print(f"Training Error Rate: {train_error:.4f}")
# print(f"Sensitivity (macro-average): {sensitivity_macro:.4f}")
# print(f"Specificity (macro-average): {specificity_macro:.4f}")
# print(f"AUC (training): {auc_train:.4f}")

# --------------------------------------------------
# 7. TEST DATA performance
# --------------------------------------------------
y_test_pred = knn_opt.predict(X_test_scaled)
test_error = 1 - accuracy_score(y_test, y_test_pred)

# print("\n--- TEST DATA PERFORMANCE ---")
# print(f"Estimated Test Error Rate: {test_error:.4f}")

# plot test error vs K
# plt.plot(k_range, test_errors, marker="o")
# plt.xlabel("K")
# plt.ylabel("Test Error Rate")
# plt.title("KNN Test Error vs K")
# plt.show()
